In [1]:
from tensorflow import keras
import numpy as np
import os
from seismic_purifier.config import BATCH_SIZE
from seismic_purifier.representation_learning_models import (
    RepresentationLearningSingleAutoencoder,
    RepresentationLearningDenoisingSingleAutoencoder,
    RepresentationLearningMultipleAutoencoder
)

2025-06-28 00:02:45.710875: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-28 00:02:45.775152: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-28 00:02:45.775191: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-28 00:02:45.775234: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-28 00:02:45.783838: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-28 00:02:45.784733: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# ============================
# 1. Configuration
# ============================
STATION_NAME='KRBG'

# Paths to your data
TRAIN_DATA_PATH = f"/home/ege/rawcovar_data/2020-09-09/processed_data/{STATION_NAME}.npy"

# Directory to save checkpoints and the final model
CHECKPOINT_DIR = 'checkpoints'
MODEL_SAVE_PATH = f'checkpoints/2020-09-09/KO.{STATION_NAME}_representation_single_5epochs.h5'


# Training parameters
EPOCHS = 5
LEARNING_RATE = 1e-3

In [3]:
# Ensure directories exist
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

# ============================
# 2. Data loading
# ============================
X_train = np.load(TRAIN_DATA_PATH)  # Expected shape: (num_samples, 3000, 3)
print(f"Training data shape: {X_train.shape}")

Training data shape: (688, 3000, 3)


In [4]:
print(np.any(np.isnan(X_train)))
np.isnan(X_train).sum()

False


0

In [5]:
# ============================
# 3. Representation Learning Model Instantiation
# ============================

# Choose the model you want to train
# For example, using RepresentationLearningSingleAutoencoder
model = RepresentationLearningSingleAutoencoder(
    name="rep_learning_autoencoder"
)

# Alternatively, you can choose other models:
#model = RepresentationLearningDenoisingSingleAutoencoder(
 #    name="rep_learning_denoising_autoencoder",
  #   input_noise_std=1e-6,
   #  denoising_noise_std=2e-1
 #)
#model = RepresentationLearningMultipleAutoencoder(
 #    name="rep_learning_autoencoder_ensemble",
  #   input_noise_std=1e-6,
   #  eps=1e-27
#)

In [6]:
model.name

'rep_learning_autoencoder'

In [7]:
# ============================
# 4. Model Compilation
# ============================
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE) 
model.compile(optimizer=optimizer)

In [8]:
# ============================
# 5. Callbacks Setup
# ============================
# Define callbacks for saving checkpoints, early stopping.
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(CHECKPOINT_DIR, 'autoencoder_epoch_{epoch:02d}.h5'),
        save_weights_only=True,
        save_freq='epoch',
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=2,
        restore_best_weights=True,
        verbose=1
    )
]

In [9]:
# ============================
# 6. Training the Representation Learning Model and Save.
# ============================
fit_result = model.fit(X_train, 
                       epochs=EPOCHS, 
                       batch_size=BATCH_SIZE, 
                       callbacks=callbacks, 
                       shuffle=False)

model.save_weights(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")

Epoch 1/5
3/3 [==============================] - ETA: 0s - loss: 1.3098
Epoch 1: saving model to checkpoints/autoencoder_epoch_01.h5
3/3 [==============================] - 32s 2s/step - loss: 1.3098
Epoch 2/5
3/3 [==============================] - ETA: 0s - loss: 1.1668
Epoch 2: saving model to checkpoints/autoencoder_epoch_02.h5
3/3 [==============================] - 4s 1s/step - loss: 1.1668
Epoch 3/5
3/3 [==============================] - ETA: 0s - loss: 1.0928
Epoch 3: saving model to checkpoints/autoencoder_epoch_03.h5
3/3 [==============================] - 5s 1s/step - loss: 1.0928
Epoch 4/5
3/3 [==============================] - ETA: 0s - loss: 1.0539
Epoch 4: saving model to checkpoints/autoencoder_epoch_04.h5
3/3 [==============================] - 4s 1s/step - loss: 1.0539
Epoch 5/5
3/3 [==============================] - ETA: 0s - loss: 1.0379
Epoch 5: saving model to checkpoints/autoencoder_epoch_05.h5
3/3 [==============================] - 5s 1s/step - loss: 1.0379
Model sav